In [1]:
#import libraries
import sys
import os
import logging
import time
import datetime as dt
import logging
import numpy as np
import pandas as pd
import norgatedata as ng
import matplotlib.pyplot as plt
from datetime import date, timedelta
from functions import trend_filter, position_size_atr, signal_filter, select_positions_2, \
                      exposure_summary, process_universe_factors, filter_ranks

sys.path.append(os.path.abspath('../fin_data'))
from utils import postgresql_conn
from trade_logic.indicators import stock_beta, stock_volatility, atr_single, sma, atr, ewma
from utils.date_functions import get_business_days, annual_base, last_business_day, time_elapsed
from utils.postgresql_data_query import get_bars, get_bm_bars, get_effective_dates, get_ranks
from data_updater.norgate_data import NorgateData

start_time = time.time()

[2024-09-09 01:59:22.338530] INFO: Norgate Data: NorgateData package v1.0.74: Init complete
connected to: dbmaster


In [2]:
## set up parameters
bm_col = '$SPX'
ccy_col = 'USD'
lo_rank = 50
hi_rank = 50
w_offset = 0
d_offset = max(1, w_offset * 7)
beta_n = 36
vola_n = 20
atr_n = 20
ewma_stop = 128
ewma_n = 128
dgt = 3
cols = ['Date','Open','High','Low','Close']
factor_cols = ['ticker','factor_date','factor_definition_id','rank']
model_factors = [85, 86, 107, 7]
account_value = 600370  # Account value: 2024-08-27
risk_factor = 0.001
sector_max = 0.20
max_positions = 200

excluded_tickers = ['VZIO']

path = 'C:/Users/VadimKovshov/Dropbox/PYTHON/CSV_Data/momentum_model/'

use_postgresql = True
end = last_business_day(offset=d_offset)
start = last_business_day(offset=d_offset + 400)
current_date = dt.date.today()
print(f'Historical range: {end.strftime("%Y-%m-%d")} to {start.strftime("%Y-%m-%d")}')

Historical range: 2024-09-06 to 2023-08-04


In [3]:
# get universe factors for a given week
eff_date, pr_date = get_effective_dates(offset_0=w_offset)
print(f'Factor dates now: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')
ranks_now = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                     hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

# get universe factors for one week prior
eff_date, pr_date = get_effective_dates(offset_0=w_offset+1)
print(f'Factor dates prior: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')
ranks_prior = process_universe_factors(eff_date=eff_date, pr_date=pr_date, lo_rank=lo_rank,
                                       hi_rank=hi_rank, model_factors=model_factors, factor_cols=factor_cols)

Factor dates now: 2024-09-06, 2024-08-30
Test universe: 1735
Factor dates prior: 2024-08-30, 2024-08-23
Test universe: 1772


In [4]:
# Filter the DataFrames
filtered_df_now = filter_ranks(ranks_now)
filtered_df_prior = filter_ranks(ranks_prior)

# Merge the two DataFrames on 'ticker' to find common and unique records
combined_df = pd.merge(filtered_df_now, filtered_df_prior[['ticker']], on='ticker', how='left', indicator=True)

# Assign labels based on the merge result
combined_df['label'] = combined_df['_merge'].map({'left_only': 'new', 'both': 'old'})

# Drop the '_merge' column as it's no longer needed
combined_df = combined_df.drop(columns=['_merge'])

# Ensure all columns from filtered_df_now are present, including the label column
columns_to_keep = list(filtered_df_now.columns) + ['label']
combined_df = combined_df[columns_to_keep]

# Optionally, reset the index
combined_df = combined_df.reset_index(drop=True)

# Filter tickers and limit to max_positions
tickers = (
    combined_df
    .loc[~combined_df['ticker'].isin(excluded_tickers), 'ticker']
    .unique()
    .tolist()[:max_positions]
)

print(f'Filtered tickers: {len(tickers)}')

Filtered tickers: 90


In [5]:
# get industry data
ng_obj = NorgateData(symbols= tickers, database='US Stocks')
ng_obj.process_metadata()
industry_df = ng_obj.df_result['metadata'].rename(columns={'symbol':'ticker', 'security_name':'name', 'GICS_level_1': 'sector', 'GICS_level_2': 'industry_group',
                                                            'GICS_level_3': 'industry', 'GICS_level_4': 'sub_industry'}).reset_index(drop=True)
industry_df = industry_df[['ticker', 'sector', 'industry_group', 'industry', 'sub_industry', 'name']]

sorted_df = pd.merge(combined_df, industry_df, on='ticker', how='left')

In [6]:
# get benchmark data
if use_postgresql:
    bm_df = get_bm_bars(bm_col,start,end) # uses saved PostgreSQL data
else:
    bm_df = ng_obj.daily_bars('$SPX', start, end, interval='D') # uses NorgateData data
    bm_df = bm_df[cols]
    bm_df['Ticker'] = bm_col

print(f'Benchmark data: {len(bm_df)}')

Benchmark data: 275


In [7]:
exposure = trend_filter(bm_df, _c='Close')
exposure = exposure[-1]
print(f'Exposure: {exposure}')

Exposure: 0.75


In [8]:
# get stock data and process signals
df_f = pd.DataFrame()
i = 0
for t in tickers:
    i += 1
    try:
        if use_postgresql:
            st_df = get_bars([t],start,end,backtest=False) # uses saved PostgreSQL data
        else:
            st_df = ng_obj.daily_bars(t, start, end, interval='D') # uses NorgateData data
            st_df = st_df[cols]
            st_df['Ticker'] = t

        beta_value = round(stock_beta(df=st_df, _c='Close', bm_df=bm_df, bm='Close', n=beta_n),dgt)
        vola_value = stock_volatility(df=st_df, _c='Close', n=vola_n, annualized=False)
        atr_value = round(atr(df=st_df, _h='High', _l='Low', _c='Close', n=atr_n).iloc[-1],2)
        ewma_value = round(ewma(df=st_df, _c='Close', _n=ewma_stop, _mp=ewma_stop).iloc[-1],dgt)
        last_close = st_df['Close'].iloc[-1]
        signal_value = signal_filter(df=st_df, _h='High', _l='Low', _c='Close', n=ewma_n, pct_change_threshold=0.15, period=90)
        atr_position_size = signal_value * position_size_atr(account_value, risk_factor, exposure, atr_value) // 10 * 10
        dollar_position = atr_position_size * st_df['Close'].iloc[-1]
        pct_position = round(dollar_position / account_value,dgt)

        data = {
            'date': st_df['Date'].iloc[-1],
            'ticker': t,
            'beta': beta_value,
            'volatility': vola_value,
            'atr': atr_value,
            'last_close': last_close,
            'ewma_stop': ewma_value,
            'signal': signal_value,
            'atr_position': atr_position_size,
            'dollar_position': dollar_position,
            'pct_position': pct_position
        }
        df = pd.DataFrame([data])
        df_f = pd.concat([df_f,df],ignore_index=True)
        if i % 20 == 0:
            print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')
    except Exception as e:
        print(f"Error processing {t}: {e}")  # Print out the exception for debugging
        continue

print(f'Completed: {i}, {t}, {time_elapsed(start_time)}')

Completed: 20, UFPT, 0 minutes and 30 seconds
Completed: 40, HTLF, 0 minutes and 38 seconds
Completed: 60, CLBT, 0 minutes and 46 seconds
Completed: 80, MSI, 0 minutes and 54 seconds
Completed: 90, CL, 0 minutes and 58 seconds


In [9]:
# final filter
df_f_final = df_f[df_f['signal'] == 1].reset_index(drop=True).copy()
df_f_final = pd.merge(df_f_final, sorted_df, on='ticker', how='left')
df_f_final = df_f_final.drop(columns=['factor_date']).reset_index(drop=True)

In [10]:
selected_positions_df = select_positions_2(df_f_final, max_exposure=exposure, max_sector_allocation=sector_max)
selected_positions_df.reset_index(drop=True).to_clipboard()
print(f'Completed: {time_elapsed(start_time)}')

Completed: 0 minutes and 58 seconds


In [11]:
styled_summary = exposure_summary(selected_positions_df)
styled_summary

Total exposure: 0.74


In [12]:
selected_positions_df.to_clipboard(index=False)
trade_df = selected_positions_df[['date','ticker','atr_position','dollar_position','ewma_stop','name']]

In [13]:
# save output to csv
selected_positions_df.to_csv(f'{path}momo_model_output_{current_date}.csv', index=False)
trade_df.to_csv(f'{path}momo_trade_file_{current_date}.csv', index=False)

print(trade_df)

         date ticker  atr_position  dollar_position  ewma_stop   
13 2024-09-06   MPWR          10.0           8063.1    783.961  \
28 2024-09-06   ENVA         200.0          15456.0     68.953   
32 2024-09-06    FBP         900.0          18324.0     18.839   
34 2024-09-06     CL         340.0          36672.4     95.969   
0  2024-09-06   GKOS         130.0          16806.4    112.753   
1  2024-09-06   PIPR          70.0          18150.3    229.438   
2  2024-09-06   CSWI          50.0          14250.0    272.359   
3  2024-09-06   FICO          10.0          17340.1   1480.313   
4  2024-09-06    KGC        1600.0          13328.0      7.841   
5  2024-09-06    IRM         210.0          22990.8     92.558   
6  2024-09-06    NWG        2250.0          19507.5      8.079   
7  2024-09-06    AEM         230.0          17684.7     69.175   
8  2024-09-06   TIGO        1000.0          26360.0     23.328   
9  2024-09-06    JEF         370.0          20764.4     50.134   
10 2024-09